# Describing the inverted index

The first thing that we're going to be doing is setting up an inverted index. This index will be used to map the term frequency to a particular document in a nested dictionary structure. 

```
index = {
    term : {
        document_id: term frequency
    }
}
```

We're also going to create a dictionary that will hold the metadata of our documents

```
documents = {
    document_id: {
        name: name of the document,
        magnitude: length of the vector # This will be important later. 
    }
}
```

In [1]:
index = {}
documents = {}

# Ingestion of documents

Now we're going to establish the procedure for document ingestion. This will take in a document and will add it to our index. 

First we're going to read in a short article from wikipedia

In [2]:
text = open("articles/austen-emma.txt", 'r', encoding='utf-8').read()
text[:1000]

"[ Emma by Jane Austen 1816 ] VOLUME I CHAPTER I Emma Woodhouse , handsome , clever , and rich , with a comfortable home and happy disposition , seemed to unite some of the best blessings of existence ; and had lived nearly twenty - one years in the world with very little to distress or vex her . She was the youngest of the two daughters of a most affectionate , indulgent father ; and had , in consequence of her sister ' s marriage , been mistress of his house from a very early period . Her mother had died too long ago for her to have more than an indistinct remembrance of her caresses ; and her place had been supplied by an excellent woman as governess , who had fallen little short of a mother in affection . Sixteen years had Miss Taylor been in Mr . Woodhouse ' s family , less as a governess than a friend , very fond of both daughters , but particularly of Emma . Between _them_ it was more the intimacy of sisters . Even before Miss Taylor had ceased to hold the nominal office of gove

For this text we're going to have to go through a few different steps in order to ingest it properly.

1. Tokenize the text
2. Calculate the frequency of the tokens
3. add the terms and their frequencies to the inverted index
4. calculate the magnitude of the document's term vector
5. add the document's metadata to the documents dictionary

We're calculating the magnitude of the term vector now since it is easier to calculate it now rather than at the time of ranking

In [3]:
from typing import List
import re

# Using a simple regular expression in order to tokenize.
def tokenize(text: str) -> List[str]:
    return re.findall(r"\b\w+\b", text.lower())

In [4]:
tokens = tokenize(text)
tokens[:5]

['emma', 'by', 'jane', 'austen', '1816']

In order to count the frequencies we're going to use the collections class from counter which does exactly that and returns a dictionary like object that we can use to reference the term frequencies

In [5]:
from collections import Counter

term_frequencies = Counter(tokens)
term_frequencies.most_common(5)

[('to', 5239), ('the', 5201), ('and', 4896), ('of', 4291), ('i', 3178)]

Now for each unique term from the text we need to add it to the index by reference of a document id like so

In [6]:
doc_id = len(documents) # By taking the len(documents) we get an incrementing identifier

for term, term_freq in term_frequencies.items():
    if term not in index:
        index[term] = {}
    index[term][doc_id] = term_freq

Now let's go ahead and calculate the magnitude of the term vector

In [8]:
from math import sqrt
mag = sqrt(sum([x**2 for x in term_frequencies.values()]))
mag

14364.35230005168

Now that we have the magnitude let's go ahead and add it as part of the metadata in our documents dictionary

In [9]:
documents[doc_id] = {
    "name": "Mariembourg.txt",
    "magnitude": mag
}

In [10]:
documents

{0: {'name': 'Mariembourg.txt', 'magnitude': 14364.35230005168}}

Now that we have the procedure established let's go ahead and turn them into functions that can take in some text and automatically add them to the index 

In [11]:
def index_document(text:str, name:str):
    tokens = tokenize(text)
    term_frequencies = Counter(tokens)
    doc_id = len(documents)

    for term in term_frequencies:
        if term not in index:
            index[term] = {}
        index[term][doc_id] = term_frequencies[term]
    
    mag = sqrt(sum([x**2 for x in term_frequencies.values()]))

    documents[doc_id] = {
        "name": name,
        "magnitude": mag
    }

In [18]:
# Getting files from the articles folder. This will be created from the data_pull.py file

from glob import glob
paths = glob("articles/*.txt")

# resetting the index and documents dicts as to not re-read data
index = {}
documents = {}

for path in paths:
    text = open(path, 'r').read()
    index_document(text, name=path)

# Searching the index

Now that we have a variety of documents ingested into the index we can start to query our index. To do this we're going to be calculating the cosine similarity of the query to the document within the query. This will give us a rank for each document's similairty to the query and in turn our search results. 

To do this we're going to 

* iterate over the terms of the query
* find the documents associated with the current term
* add the tfidf scores for each of the individual documents
* normalize the sum of the tfidf scores using the pre-calculated magnitude of the documents term vector
* get the top 10 docuemnt ids from the calculated scores

In [31]:
from collections import Counter
from math import log

query = "Flambeau priest"
query_terms = tokenize(query)

N = len(documents) # The number of documents in the corpus
scores = Counter() # Counter to hold the scores and default to zero if it doesn't exist

for term in query_terms:
    df = len(index[term]) # The document freqeuncy for the term 
    idf = log(N/df) # The inverse-document frequency for the term

    for doc_id, tf in index[term].items():
        scores[doc_id] += (tf * idf) # adding the tfidf to the document's score for this query

articles\chesterton-brown.txt : 0.06230762498080422
articles\bible-kjv.txt : 0.0031629507437594714
articles\blake-poems.txt : 0.0016854240823057666
articles\shakespeare-hamlet.txt : 0.0007197148531950618
articles\chesterton-ball.txt : 0.0004371842305225936
articles\melville-moby_dick.txt : 0.0002783200568285072
articles\whitman-leaves.txt : 0.000253875114132975
articles\chesterton-thursday.txt : 0.00020280507834880357
articles\milton-paradise.txt : 0.00017685509193384998
articles\bryant-stories.txt : 0.00011469704186578875


In [ ]:
# Normalize the scores based on the magnitude we calculated during document ingestion
for doc_id, score in scores.items():
    scores[doc_id] = score / documents[doc_id]['magnitude']

In [ ]:
# get the top results from the query
for doc_id, score in scores.most_common(10):
    print(documents[doc_id]['name'], ":", score)

In [32]:
def search(query:str):
    query_terms = tokenize(query)

    N = len(documents) # The number of documents in the corpus
    scores = Counter() # Counter to hold the scores and default to zero if it doesn't exist

    for term in query_terms:
        df = len(index[term]) # The document freqeuncy for the term 
        idf = log(N/df) # The inverse-document frequency for the term

        for doc_id, tf in index[term].items():
            scores[doc_id] += (tf * idf) # adding the tfidf to the document's score for this query
    # Normalize the scores based on the magnitude we calculated during document ingestion
    for doc_id, score in scores.items():
        scores[doc_id] = score / documents[doc_id]['magnitude']

    # get the top results from the query
    for doc_id, score in scores.most_common(10):
        print(documents[doc_id]['name'], ":", score)

In [35]:
search("priest")

articles\chesterton-brown.txt : 0.010797435480809386
articles\bible-kjv.txt : 0.0031629507437594714
articles\blake-poems.txt : 0.0016854240823057666
articles\shakespeare-hamlet.txt : 0.0007197148531950618
articles\chesterton-ball.txt : 0.0004371842305225936
articles\melville-moby_dick.txt : 0.0002783200568285072
articles\whitman-leaves.txt : 0.000253875114132975
articles\chesterton-thursday.txt : 0.00020280507834880357
articles\milton-paradise.txt : 0.00017685509193384998
articles\bryant-stories.txt : 0.00011469704186578875
